In [7]:
import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_PROVIDER") or "openai"

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

current_dir = Path.cwd()
data_dir = current_dir.parent / "data"
index_dir = current_dir.parent / "index"


In [8]:
type(data_dir)

pathlib.PosixPath

In [9]:
# Loader
loader = PyPDFLoader(os.path.join(str(data_dir), "sample.pdf"))
pages = loader.load_and_split()

#print(pages)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50,
)

texts = text_splitter.split_documents(pages)
texts 

[Document(metadata={'producer': 'Skia/PDF m125', 'creator': 'Chromium', 'creationdate': '2024-07-03T03:24:25+00:00', 'title': '경제 : 네이버 뉴스', 'moddate': '2024-07-03T03:24:25+00:00', 'source': '/Users/jurepi/Work/Study/jo-langchain/data/sample.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='에코프로  공급물량  신청  사이트 ?…" 사\n기입니다 "\n"60 세  이상  고용률  높이면  잠재성장률\n증가 "… 노동시장  유연성  전제돼야\n\'오염물질 · 온실가스  무배출 \' 수소차  첫\n등록  5 년  만에  1000 대  돌파\n다시  불어온  청약  광풍 … 하루에  17 만명\n몰려'),
 Document(metadata={'producer': 'Skia/PDF m125', 'creator': 'Chromium', 'creationdate': '2024-07-03T03:24:25+00:00', 'title': '경제 : 네이버 뉴스', 'moddate': '2024-07-03T03:24:25+00:00', 'source': '/Users/jurepi/Work/Study/jo-langchain/data/sample.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='몰려\n롯데바이오 , 송도에  바이오의약품  생산\n플랜트  착공 … 경제효과  7.6 조원\n대기업  CEO 들도  \' 고령화 \'… 서울대  출\n신은  줄었다\n“엄청난  회복세 ”… 테슬라 , 주가  10%\n넘게  급등\n이복현  금감원장  " 증권사 , 손쉬운  수익'),
 Document(metadata={'producer': 'Skia/PDF

In [ ]:
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

vectorstore = Chroma.from_documents(texts, embeddings_model)

chroma_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k':1, 'fetch_k':4}
)


In [11]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2

print("type of bm25", type(bm25_retriever))

type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.2, 0.8]
)


In [13]:
query = "에코프로에 대해서 알려줘"

In [14]:
docs = ensemble_retriever.invoke(query)

In [15]:
def print_docs(docs):
    print("="*100)    
    for doc in docs:
        print(doc.page_content)
        print("-"*100)

In [16]:
print_docs(docs)

이  콘텐츠의  저작권은  저작권자  또는  제공처에  있으며 , 이를  무단  이용하는  경우  저작권법  등에  따라  법적  책임을  질  수  있습니다 .
----------------------------------------------------------------------------------------------------
SBS Biz
 아이뉴스 24
로그인 전체서비스
서비스안내 오류신고 고객센터
기사배열  책임자  : 김수향 청소년  보호  책임자  : 이정규
01 55
02 09
02 03
----------------------------------------------------------------------------------------------------
에코프로  공급물량  신청  사이트 ?…" 사
기입니다 "
"60 세  이상  고용률  높이면  잠재성장률
증가 "… 노동시장  유연성  전제돼야
'오염물질 · 온실가스  무배출 ' 수소차  첫
등록  5 년  만에  1000 대  돌파
다시  불어온  청약  광풍 … 하루에  17 만명
몰려
----------------------------------------------------------------------------------------------------


In [17]:


from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2
print("type of bm25", type(bm25_retriever))

type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [18]:
docs

[Document(metadata={'producer': 'Skia/PDF m125', 'creator': 'Chromium', 'creationdate': '2024-07-03T03:24:25+00:00', 'title': '경제 : 네이버 뉴스', 'moddate': '2024-07-03T03:24:25+00:00', 'source': '/Users/jurepi/Work/Study/jo-langchain/data/sample.pdf', 'total_pages': 4, 'page': 3, 'page_label': '4'}, page_content='이  콘텐츠의  저작권은  저작권자  또는  제공처에  있으며 , 이를  무단  이용하는  경우  저작권법  등에  따라  법적  책임을  질  수  있습니다 .'),
 Document(metadata={'producer': 'Skia/PDF m125', 'creator': 'Chromium', 'creationdate': '2024-07-03T03:24:25+00:00', 'title': '경제 : 네이버 뉴스', 'moddate': '2024-07-03T03:24:25+00:00', 'source': '/Users/jurepi/Work/Study/jo-langchain/data/sample.pdf', 'total_pages': 4, 'page': 2, 'page_label': '3'}, page_content='SBS Biz\n 아이뉴스 24\n로그인 전체서비스\n서비스안내 오류신고 고객센터\n기사배열  책임자  : 김수향 청소년  보호  책임자  : 이정규\n01\x0055\n02\x0009\n02\x0003'),
 Document(metadata={'moddate': '2024-07-03T03:24:25+00:00', 'creationdate': '2024-07-03T03:24:25+00:00', 'page_label': '1', 'title': '경제 : 네이버 뉴스', 'producer': 'Skia/P

In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model=model_name, temperature=0)

def format_docs(docs):
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted

rag_chain = (
    {
        "context": ensemble_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)



In [21]:
rag_chain.invoke(query)

'에코프로는 주로 친환경 및 신재생 에너지 관련 사업을 하는 한국의 기업입니다. 특히 배터리 소재, 친환경 소재 개발 등에 집중하고 있으며, 전기차 배터리용 양극재 등 첨단 소재를 생산하는 것으로 알려져 있습니다. 최근에는 공급물량 신청과 관련된 사기 사례가 언급되기도 했으니, 에코프로와 관련된 거래나 신청 시에는 공식 채널을 통해 신중하게 확인하는 것이 중요합니다.'